In [1]:
import numpy as np
import pandas as pd
import zipfile
from zipfile import ZipFile
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import os

import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import EarlyStopping
import os
print(os.listdir("../input"))

Using TensorFlow backend.


['train.csv', 'sample_submission.csv', 'test.csv']


In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')
sub = pd.read_csv('../input/sample_submission.csv')

In [61]:
train.shape,test.shape

((1000, 45), (1000, 2))

In [62]:
num_words = 20000
max_len = 220
emb_size = 256
num_model = 3

In [63]:
tokenizer=Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(list(train['comment_text']) + list(test['comment_text']))
Y=np.where(train['target']>=0.5,1,0)

In [64]:
print(tokenizer.document_count)

2000


In [65]:
X = tokenizer.texts_to_sequences(list(train['comment_text']))
test_X = tokenizer.texts_to_sequences(list(test['comment_text']))

In [66]:
X = sequence.pad_sequences(X, maxlen = max_len)
test_X = sequence.pad_sequences(test_X, maxlen = max_len)

In [79]:
# create the model
embedding_vecor_length = emb_size
model = Sequential()
model.add(Embedding(num_words, embedding_vecor_length, input_length=X.shape[1]))
model.add(Dropout(0.25))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 220, 256)          5120000   
_________________________________________________________________
dropout_19 (Dropout)         (None, 220, 256)          0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 128)               197120    
_________________________________________________________________
dropout_20 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 129       
Total params: 5,317,249
Trainable params: 5,317,249
Non-trainable params: 0
_________________________________________________________________
None


In [80]:
score=[]
predictions=[]
for i in range(num_model):
    X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.20)
    es = EarlyStopping(monitor='val_loss', patience=100)
    model.fit(X_train,Y_train, validation_data=(X_test, Y_test), epochs=3, batch_size=1024,callbacks=[es])
    pred = model.predict(test_X)
    scores = model.evaluate(test_X, pred, verbose=0)
    score.append(scores)
    predictions.append(pred)
    print(i)

Train on 800 samples, validate on 200 samples
Epoch 1/3
800/800 [==============================] - 3s 4ms/step - loss: 0.6897 - acc: 0.6313 - val_loss: 0.6754 - val_acc: 0.9200
Epoch 2/3
800/800 [==============================] - 1s 758us/step - loss: 0.6728 - acc: 0.9350 - val_loss: 0.6587 - val_acc: 0.9200
Epoch 3/3
800/800 [==============================] - 1s 747us/step - loss: 0.6525 - acc: 0.9450 - val_loss: 0.6372 - val_acc: 0.9200
0
Train on 800 samples, validate on 200 samples
Epoch 1/3
800/800 [==============================] - 1s 750us/step - loss: 0.6282 - acc: 0.9388 - val_loss: 0.5961 - val_acc: 0.9450
Epoch 2/3
800/800 [==============================] - 1s 747us/step - loss: 0.5945 - acc: 0.9388 - val_loss: 0.5474 - val_acc: 0.9450
Epoch 3/3
800/800 [==============================] - 1s 760us/step - loss: 0.5446 - acc: 0.9388 - val_loss: 0.4730 - val_acc: 0.9450
1
Train on 800 samples, validate on 200 samples
Epoch 1/3
800/800 [==============================] - 1s 755us/

In [81]:
score=pd.DataFrame(score)
score[0].mean()

0.4936319772402445

In [82]:
fina_pred=np.average(predictions,axis=0)
fina_pred=pd.DataFrame(fina_pred)

In [83]:
ids=test['id']
ids=pd.DataFrame(ids)

In [84]:
submission=pd.concat([ids,fina_pred],axis=1)
submission['prediction']=submission.iloc[:,1:]
submission=submission.drop([0],axis=1)
submission.to_csv("submission.csv",index=False)